In [2]:
# Packages 

import numpy as np

from scipy import linalg, interpolate
from numba import jit
import matplotlib.pyplot as plt
import time
import copy


In [3]:
# Parameters

delta=0.025
alpha=0.11
beta=0.981
rhoa=0.90
sigma=1

nk=250
nz=2


In [4]:

kgrid=np.linspace(0,30,nk)[:,None]

zgrid=np.array([0.6,1])[:,None]

Pz=np.array([[0.8,0.2],[0.05,0.95]])


In [5]:

@jit(nopython=True)
def ergdisc(P): # finding ergodic distribution of markov process

    eP = np.linalg.eig(P.transpose())
    uz=np.real(eP[1])
    uzi=np.where(np.real(eP[0])==1)
    f=uz[:,uzi[0][0]]/sum(uz[:,uzi[0][0]])
    return(f)


In [6]:

zerg=ergdisc(Pz)[:,None]

N=sum(zgrid*zerg)


In [100]:
r0=0.02

Kd=((r0+delta)/(alpha*N**(1-alpha)))**(1/(alpha-1)) # capital demand

w=(1-alpha)*(Kd**alpha)*N**(-alpha) # wage

y=kgrid*(1+r0)+w*zgrid.transpose()

r=copy.copy(r0)


In [117]:
# solve consumption policy

cpol=copy.copy(y)

dff=10

iter=0

while (dff>10**(-6)) &(iter<1000):
    
    iter=iter+1
    cstar=((cpol @ Pz.transpose())**(-sigma)*beta*(1+r))**(-(1/sigma))

    kminus=(kgrid+cstar-w*zgrid.transpose())/(1+r)

    cpolold=copy.copy(cpol)

    for s in range(0,nz):
        kmin=kminus[0,s]
        splinein=np.concatenate([kminus[:,[s]],cstar[:,[s]]],axis=1)
        splinein=splinein[splinein[:,0].argsort()]
        cpolspline=interpolate.CubicSpline(splinein[:,0],splinein[:,1]) # consumption policy    
        bcind=kgrid<kmin # borrowing constrain indicator
        cpol[:,[s]]=cpolspline(kgrid)*(1-bcind)+bcind*y[:,[s]] # consumption policy on grid
    
    dff=max((cpolold-cpol).flatten())

kpol=kgrid*(1+r)+w*zgrid.transpose()-cpol # capital choice

In [169]:
# capital transition matrix
klowind=(kpol[:,[0]]>=kgrid.transpose())
klowind=(klowind*1).sum(axis=1)-1
khighind=klowind+1   

klowwgt=1-(kpol[:,[0]]-kgrid[klowind])/(kgrid[khighind]-kgrid[klowind])

array([[1.        ],
       [0.64554858],
       [0.93552969],
       [0.12612628],
       [0.26752807],
       [0.37952617],
       [0.47096756],
       [0.54773639],
       [0.61308592],
       [0.66959493],
       [0.71883774],
       [0.76219077],
       [0.80053723],
       [0.8347051 ],
       [0.86523847],
       [0.89267683],
       [0.91737806],
       [0.93971403],
       [0.95993212],
       [0.97829937],
       [0.99499371],
       [0.01021277],
       [0.02408896],
       [0.03677182],
       [0.04836279],
       [0.05897699],
       [0.06869406],
       [0.07760385],
       [0.08577021],
       [0.09326402],
       [0.10013735],
       [0.10644672],
       [0.11223511],
       [0.117548  ],
       [0.12242123],
       [0.12689174],
       [0.13098963],
       [0.13474515],
       [0.13818368],
       [0.14133024],
       [0.14420631],
       [0.1468327 ],
       [0.14922768],
       [0.15140867],
       [0.15339123],
       [0.15519004],
       [0.15681838],
       [0.158

In [179]:
kgrid[[1,2,249]]

array([[ 0.12048193],
       [ 0.24096386],
       [30.        ]])